Going to model the data here.

In [104]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy.optimize import Bounds
from plotdf import plotdf

data = pd.read_excel('data.xlsx')

time = np.linspace(1, 24, 24)
hbin = 2.344E-6
hpin = hbin
hpx = [1, 24]
hpy = [hpin, 0]
ic = (0, 0, hpin)
ac = (1E-5)
s=1.38476998e-05
mu=6.86142089e+00

def func(x, kr):
    dB = - kr * x[0] * x[1] + s/((((x[0]-mu)**2)+1)**(3/2))
    dC = s/((((x[0]-mu)**2)+1)**(3/2))
    dP = - kr * x[0] * x[1]
    return(dB, dC, dP)

def err(params):
    kr = params
    ODEoutputB = odeint(func, ic, time, args = (kr))
    ODEoutputP = odeint(func, ic, hpx, args = (kr))
    errB = (ODEoutputB[:,0] - data["hpadj mol/dL"])**2
    errP = (ODEoutputP[:,1] - hpy)**2
    errC = (ODEoutputB[:,2] - data["cadj mol/dL"])**2
    return (np.sum(errB) + np.sum(errC) + np.sum(errP))

bounds = ([0, 10E100])
sol = minimize(err, x0 = ac, bounds = bounds, method = "Nelder-Mead")
print(sol)

fitdata = odeint(func, ic, data["time"], args=(sol.x[0]))
plt.scatter(data["time"], data['hpadj mol/dL'], label = "Experimental data - hemoglobin")
plt.scatter(data["time"], data['cadj mol/dL'], label = "Experimental data - hemoglobin - control group")
plt.scatter(hpx, hpy, label = "Assumed values - haptoglobin")
plt.plot(data["time"], fitdata[:,0], label = "Estimated hemoglobin")
plt.plot(data["time"], fitdata[:,2], label = "Estimated hemoglobin - control group")
plt.plot(data["time"], fitdata[:,1], label = "Estimated haptoglobin")
plt.xlabel("Time (min)")
plt.ylabel("Concentration (mol/dL)")
plt.legend()

TypeError: 'int' object is not iterable

In [ ]:
# plotdf(func, # Function giving the rhs of the diff. eq. system
#      np.array([0,200]), # [xmin,xmax]
#      np.array([0,200]),# [ymin,ymax]
#      [(0,150)], # list of initial values for trajectories (optional)
#      # Additional parameters for `f` (optional)
#      parameters={"GB":50,"kr":0.3,"GP":0.01},
#      gridsteps=20,
#      nsteps = 10000,
#      tmax=20)

# plt.xlabel("B, concentration of hemoglobin (g/dL)")
# plt.ylabel("P, concentration of haptoglobin (g/dL)")